# Hopsworks Feature Store - Part 01 - Load, Engineer & Connect

This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, we will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with Hopworks' feature store for batch data with a goal of training and deploying a model that can predict fraudulent transactions.

**This notebook is divided in 3 sections:** 
1. Loading the data and feature engineeing,
2. Connect to the Hopsworks feature store,
3. Create feature groups and upload them to the feature store.

![tutorial-flow](images/load_engineer_connect.png)

First of all we will load the data and do some feature engineering on it.

### Loading the Data

The data we will use comes from three different CSV files:

- `credit_cards.csv`: credit card information such as expiration date and provider.
- `transactions.csv`: transaction information such as timestamp, location, and the amount. Importantly, the binary `fraud_label` variable tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

We can conceptualize these CSV files as originating from separate data sources.
**All three files have a credit card number column `cc_num` in common, which we can use for joins.**

Let's go ahead and load the data.

In [1]:
import pandas as pd
from hops import hdfs

credit_cards_df = pd.read_csv("hdfs:///Projects/{}/Jupyter/fraud_detection/data/credit_cards.csv".format(hdfs.project_name()))
credit_cards_df.head(3)

,cc_num,provider,expires
0,4031433455074417,visa,02/25
1,4436104537406320,visa,08/24
2,4571305563689391,visa,02/21


In [2]:
profiles_df = pd.read_csv("hdfs:///Projects/{}/Jupyter/fraud_detection/data/profiles.csv".format(hdfs.project_name()), parse_dates=["birthdate"])
profiles_df.head(3)

,name,sex,mail,birthdate,City,Country,cc_num
0,Teresa Smith,F,kevin70@yahoo.com,1972-11-07,Camarillo,US,4031433455074417
1,Luis Hays,M,kevinstewart@hotmail.com,1995-07-28,Troutdale,US,4436104537406320
2,Kyle Clark,M,davidflores@gmail.com,1954-12-30,Fort Washington,US,4571305563689391


In [1]:
trans_df = pd.read_csv(
    "hdfs:///Projects/{}/Jupyter/fraud_detection/data/transactions.csv.zip".format(hdfs.project_name()), parse_dates=["datetime"])
trans_df.head(3)

NameError: name 'pd' is not defined

### Feature Engineering

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning we will create additional features based on these patterns. In particular, we will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

In [37]:
import numpy as np

# Compute age at transaction.
age_df = trans_df.merge(profiles_df, on="cc_num", how="left")
trans_df["age_at_transaction"] = (age_df["datetime"] - age_df["birthdate"]) / np.timedelta64(1, "Y")

# Compute days until card expires.
card_expiry_df = trans_df.merge(credit_cards_df, on="cc_num", how="left")
card_expiry_df["expires"] = pd.to_datetime(card_expiry_df["expires"], format="%m/%y")
trans_df["days_until_card_expires"] = (card_expiry_df["expires"] - card_expiry_df["datetime"]) / np.timedelta64(1, "D")

trans_df[["age_at_transaction", "days_until_card_expires"]].head()

,age_at_transaction,days_until_card_expires
0,30.346961,-579.000012
1,31.058817,1856.999988
2,45.780543,1154.999977
3,52.740303,1825.999931
4,87.717065,-214.000139


Next, we create features that for each credit card aggregate data from multiple time steps.

We start by computing the distance between consecutive transactions, which we will call `loc_delta`.
Here we use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [18]:
from math import radians

# Do some simple preprocessing.
trans_df.sort_values("datetime", inplace=True)
trans_df[["longitude", "latitude"]] = trans_df[["longitude", "latitude"]].applymap(radians)

def haversine(long, lat):
    """Compute Haversine distance between each consecutive coordinate in (long, lat)."""

    long_shifted = long.shift()
    lat_shifted = lat.shift()
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    c = 2*np.arcsin(np.sqrt(a + b))

    return c


trans_df["loc_delta"] = trans_df.groupby("cc_num")\
    .apply(lambda x : haversine(x["longitude"], x["latitude"]))\
    .reset_index(level=0, drop=True)\
    .fillna(0)

Next we compute windowed aggregates. Here we will use 4-hour windows, but feel free to experiment with different window lengths by setting `window_len` below to a value of your choice.

In [21]:
window_len = "4h"
cc_group = trans_df.groupby("cc_num")

# Create dataframe for window aggregations.
window_aggs_df = pd.DataFrame()
window_aggs_df["tid"] = trans_df["tid"]

# Moving average of transaction volume.
window_aggs_df['trans_volume_mavg'] = cc_group[["datetime", "amount"]]\
    .rolling(window_len, on="datetime")\
    .mean()\
    .reset_index(level=0, drop=True)\
    .drop(columns=["datetime"])

# Moving standard deviation of transaction volume.
window_aggs_df['trans_volume_mstd'] = cc_group[["datetime", "amount"]]\
    .rolling(window_len, on="datetime")\
    .std()\
    .reset_index(level=0, drop=True)\
    .drop(columns=["datetime"])\
    .fillna(0)

# Moving average of transaction frequency.
window_aggs_df['trans_freq'] = cc_group[["datetime", "tid"]]\
    .rolling(window_len, on="datetime")\
    .count()\
    .reset_index(level=0, drop=True)\
    .drop(columns=["datetime"])

# Moving average of location difference between consecutive transactions.
window_aggs_df["loc_delta_mavg"] = cc_group[["datetime", "loc_delta"]]\
    .rolling(window_len, on="datetime")\
    .mean()\
    .reset_index(level=0, drop=True)\
    .drop(columns=["datetime"])

window_aggs_df.tail()

,tid,trans_volume_mavg,trans_volume_mstd,trans_freq,loc_delta_mavg
557272,8fcb8bb87f0edfe2464f49e0b3b49a31,63.97,0.0,1.0,3.204164e-06
595763,c615c36e298e4ce00238484a93b581ae,585.64,0.0,1.0,2.273392e-06
641178,f0150f187cc462f7f649a4a8e13af28f,48.72,0.0,1.0,1.973912e-06
605906,bbd562425595518ec5d97ca41400aef9,1.42,0.0,1.0,6.445220e-07
648097,1a348f9ce0477c264edd70cff9a46beb,11.82,0.0,1.0,1.550189e-06


### Creating Feature Groups

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features. In our case, we will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `tid` as primary key, which will allow us to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting we would likely want to experiment with different window lengths. In that case, we can create feature groups with identical schema for each window length. 

Before we can create a feature group we need to connect to our feature store.

In [11]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [12]:
# TODO do we really need event_time??
# Not sure if we will actually do a point in time split...,
# or if it's needed for the chronological split.

trans_fg = fs.create_feature_group(
    name="transactions",
    description="Transaction data.",
    primary_key=['tid'],
    online_enabled=True,
    event_time=['datetime']
)

Here we have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `save` function.

In [13]:
trans_fg.save(trans_df)

Configuring ingestion job...
Uploading Pandas dataframe...
Launching ingestion job...
Ingestion Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/125/jobs/named/credit_cards_1_insert_fg_25042022201841/executions


At the creation of the feature group, you will prompted with an URL that will directly link to it; there you will be able to explore some of the aspects of your newly created feature group.

[//]: <> (insert GIF here)

As can be seen from the following GIF, the feature group and all of it's metadata are now searchable within the interface; offering extended abilities for teams and collaborators to use or re-use similar feature groups for their own projects. 

[//]: <> (insert GIF here)

Let's move on and do the same thing for the feature group with our windows aggregation.

In [2]:
window_aggs_fg = fs.create_feature_group(
    name=f"transactions_{window_len}_aggs",
    description=f"Aggregate transaction data over {window_len} windows.",
    primary_key=["tid"],
    online_enabled=True,
)
window_aggs_fg.save(window_aggs_df)

NameError: name 'fs' is not defined

Both feature groups are now accessible and searchable in the UI

### Next; Part 02

In the following notebook we will use our feature groups to create a dataset we can train a model on.